### 1. LCEL의 기본 구조를 배워봅시다.
1. 프롬프트 템플릿
2. 모델
3. 출력 형태

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os 
project_name = "wanted_2nd_langchain_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

### 1. 프롬프트 템플릿 설정하기

In [2]:
from langchain_core.prompts import PromptTemplate

template = "최근 {year}년 동안 한국에 오는 관광객 수는?"
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['year'], input_types={}, partial_variables={}, template='최근 {year}년 동안 한국에 오는 관광객 수는?')

In [3]:
prompt_template.format(year=5)

'최근 5년 동안 한국에 오는 관광객 수는?'

### 2. 모델 선택하기

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [5]:
chain = prompt_template | model
chain

PromptTemplate(input_variables=['year'], input_types={}, partial_variables={}, template='최근 {year}년 동안 한국에 오는 관광객 수는?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002A02A9C3AD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002A02A9F7610>, root_client=<openai.OpenAI object at 0x000002A02A4F4650>, root_async_client=<openai.AsyncOpenAI object at 0x000002A02A9F7110>, model_name='gpt-4.1-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [6]:
input = {"year": 3}
answer = chain.invoke(input)
print(answer.content)

최근 3년 동안 한국을 방문한 관광객 수에 대해 알려드리겠습니다.

- **2021년**: 코로나19 팬데믹 영향으로 국제 관광객 수가 크게 감소하여 약 1,300,000명 수준이었습니다.
- **2022년**: 방역 완화와 함께 관광객 수가 점차 회복되어 약 5,000,000명 정도로 추정됩니다.
- **2023년**: 관광 산업 회복세가 이어지면서 약 10,000,000명 내외로 증가한 것으로 예상됩니다.

정확한 최신 통계는 한국관광공사나 통계청 공식 자료를 참고하시는 것이 좋습니다. 필요하시면 해당 기관의 최신 보고서 링크도 안내해 드릴 수 있습니다.


In [7]:
answer

AIMessage(content='최근 3년 동안 한국을 방문한 관광객 수에 대해 알려드리겠습니다.\n\n- **2021년**: 코로나19 팬데믹 영향으로 국제 관광객 수가 크게 감소하여 약 1,300,000명 수준이었습니다.\n- **2022년**: 방역 완화와 함께 관광객 수가 점차 회복되어 약 5,000,000명 정도로 추정됩니다.\n- **2023년**: 관광 산업 회복세가 이어지면서 약 10,000,000명 내외로 증가한 것으로 예상됩니다.\n\n정확한 최신 통계는 한국관광공사나 통계청 공식 자료를 참고하시는 것이 좋습니다. 필요하시면 해당 기관의 최신 보고서 링크도 안내해 드릴 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 167, 'prompt_tokens': 20, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CLKZFfrY18n74gsFD83Y4K9pvqWi4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f58995c-e4bf-4fc1-bf98-8a86a0ba5b1c-0', usage_metadata={'input_tokens': 20, 'output_tokens': 167, 'tot

### 3. 출력 양식 정해보기
- Stroutputparser
- jsonoutputparser
- pydanticoutputparser
- commaseperateoutputparser

In [8]:
from langchain_core.output_parsers import StrOutputParser

outputparser = StrOutputParser()

chain = prompt_template | model | outputparser

input = {"year": 2}
answer = chain.invoke(input)
print(answer)

최근 2년 동안 한국을 방문한 관광객 수에 대한 최신 통계는 다음과 같습니다.

- 2022년: 약 1,000만 명
- 2023년: 약 1,500만 명 (예상치 또는 잠정치)

코로나19 팬데믹으로 인해 2020년과 2021년에는 해외 관광객 수가 크게 감소했으나, 2022년부터 방역 완화와 국제 여행 재개로 관광객 수가 빠르게 회복되고 있습니다.

정확한 최신 수치는 한국관광공사(KTO)나 통계청의 공식 발표 자료를 참고하시는 것이 좋습니다. 필요하시면 해당 기관의 최신 보고서 링크도 안내해 드릴 수 있습니다.


# 실습
- 각자 알아서 질문 주제 정하고
- 프롬프트 템플릿을 좀 더 자세하게 작성해보기

In [26]:
# 템플릿을 좀 더 자세하게 설정해보기 - 1
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 
학생들을 위한 영어 연습 자료를 만들어주세요
주어진 상황에 맞게 영어 지문을 작성해주세요
양식은 [FORMAT]을 참고하여 작성해주세요

#상황:
{situation}

#FORMAT:
- 영어 지문:
- 영어 문제:
- 한글 해석:
- 정답:
"""

prompt = PromptTemplate.from_template(template)

en_chain = prompt | model | outputparser
answer = en_chain.invoke({"situation": "학교 친구들끼리 주먹질하며 싸웠다"})
print(answer)

- 영어 지문:  
Yesterday, two students at school got into a fight. They started punching each other during recess. Some friends tried to stop them, but the fight lasted for a few minutes. After the teachers arrived, the students were separated and sent to the principal’s office. Fighting is not allowed at school because it can hurt people and cause problems.

- 영어 문제:  
1. When did the fight happen?  
2. What did the friends do when the fight started?  
3. Where were the students sent after the teachers arrived?  
4. Why is fighting not allowed at school?

- 한글 해석:  
어제 학교에서 두 학생이 싸웠다. 그들은 쉬는 시간에 서로 주먹질을 시작했다. 몇몇 친구들이 싸움을 말리려고 했지만, 싸움은 몇 분간 계속되었다. 선생님들이 도착한 후, 학생들은 분리되어 교장실로 보내졌다. 학교에서는 사람을 다치게 하고 문제를 일으킬 수 있기 때문에 싸움이 허용되지 않는다.

- 정답:  
1. During recess yesterday.  
2. They tried to stop the fight.  
3. To the principal’s office.  
4. Because it can hurt people and cause problems.


In [18]:
template1 = "올해 {country}에서 {type} 트랜드중 가장 핫한 키워드는 무었인가요?"
prompt_template1 = PromptTemplate.from_template(template1)
prompt_template1

PromptTemplate(input_variables=['country', 'type'], input_types={}, partial_variables={}, template='올해 {country}에서 {type} 트랜드중 가장 핫한 키워드는 무었인가요?')

In [19]:
prompt_template1.format(country="한국", type="AI")

'올해 한국에서 AI 트랜드중 가장 핫한 키워드는 무었인가요?'

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [21]:
chain1 = prompt_template1 | model
chain1

PromptTemplate(input_variables=['country', 'type'], input_types={}, partial_variables={}, template='올해 {country}에서 {type} 트랜드중 가장 핫한 키워드는 무었인가요?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002A056CD6D50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002A056CD2690>, root_client=<openai.OpenAI object at 0x000002A055FBC310>, root_async_client=<openai.AsyncOpenAI object at 0x000002A056CD6A50>, model_name='gpt-4.1-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [24]:
input1 = {"country": "한국", "type": "AI"}
answer1 = chain1.invoke(input1)
print(answer1.content)

2024년 한국에서 AI 트렌드 중 가장 핫한 키워드는 다음과 같습니다:

1. **생성형 AI (Generative AI)**  
   ChatGPT, GPT-4, 그리고 다양한 생성형 AI 모델들이 큰 주목을 받고 있습니다. 텍스트, 이미지, 영상 생성 등 다양한 분야에서 활용도가 급증하고 있죠.

2. **AI 윤리 및 규제**  
   AI 기술의 발전과 함께 개인정보 보호, 편향성 문제, 책임 소재 등 윤리적 이슈와 관련된 논의가 활발합니다. 정부와 기업 차원에서 AI 윤리 가이드라인 마련이 중요한 키워드입니다.

3. **AI 융합 서비스**  
   AI가 금융, 의료, 제조, 교육 등 다양한 산업과 융합되어 새로운 서비스와 비즈니스 모델을 창출하는 사례가 늘고 있습니다.

4. **엣지 AI (Edge AI)**  
   클라우드가 아닌 디바이스 자체에서 AI 연산을 수행하는 엣지 AI 기술이 IoT, 스마트 팩토리 등에서 주목받고 있습니다.

5. **AI 기반 맞춤형 콘텐츠 및 마케팅**  
   개인 맞춤형 추천, 광고, 콘텐츠 생성에 AI가 적극 활용되면서 마케팅 분야에서 큰 변화가 일어나고 있습니다.

이 외에도 **대화형 AI**, **멀티모달 AI**, **AI 자동화 및 로보틱 프로세스 자동화(RPA)** 등이 한국 내 AI 트렌드에서 중요한 키워드로 꼽힙니다.


In [25]:
from langchain_core.output_parsers import StrOutputParser

outputparser = StrOutputParser()

chain1 = prompt_template1 | model | outputparser

input1 = {"country": "한국", "type": "AI"}
answer1 = chain1.invoke(input1)
print(answer1)

2024년 한국에서 AI 트렌드 중 가장 핫한 키워드는 다음과 같습니다:

1. **생성형 AI (Generative AI)**  
   ChatGPT, GPT-4, 그리고 다양한 생성형 AI 모델들이 큰 주목을 받고 있습니다. 텍스트, 이미지, 음성 등 다양한 콘텐츠를 자동으로 생성하는 기술이 산업 전반에 빠르게 확산되고 있습니다.

2. **AI 윤리 및 규제**  
   AI 기술의 발전과 함께 개인정보 보호, 편향성 문제, 투명성 확보 등 윤리적 이슈가 중요한 화두로 떠오르고 있습니다. 정부와 기업 모두 AI 윤리 기준 마련과 규제 대응에 집중하고 있습니다.

3. **AI 융합 서비스**  
   AI가 금융, 의료, 제조, 교육 등 다양한 산업과 융합되어 맞춤형 서비스와 스마트 솔루션을 제공하는 사례가 늘어나고 있습니다.

4. **엣지 AI (Edge AI)**  
   클라우드가 아닌 디바이스 자체에서 AI 연산을 수행하는 엣지 AI 기술이 IoT, 스마트폰, 자율주행 등 분야에서 주목받고 있습니다.

5. **대화형 AI 및 챗봇**  
   고객 서비스, 상담, 교육 등에서 자연스러운 대화가 가능한 AI 챗봇이 활발히 도입되고 있습니다.

이 외에도 AI 반도체, 메타버스와의 결합, AI 기반 데이터 분석 등이 중요한 트렌드로 자리잡고 있습니다.
